In [37]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from keras.models import Sequential, Model
from keras.datasets import cifar10
from keras.layers import Dense, Flatten, Reshape, BatchNormalization, Conv2D, Conv2DTranspose, Activation, Input
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam

In [38]:
base_path = './'
print(os.path.isdir(base_path))

In [39]:
%cd ./

In [40]:
#   Build Generator
def build_generator(input_shape):
    G = Sequential()
    G.add(Dense(2048, input_shape = input_shape, kernel_initializer = 'he_normal'))
    G.add(Reshape((2, 2, 512)))
    G.add(BatchNormalization())
    G.add(Activation('relu'))

    G.add(Conv2DTranspose(256, (5, 5), strides = 2, padding = 'same', kernel_initializer = 'he_normal'))
    G.add(BatchNormalization())
    G.add(Activation('relu'))

    G.add(Conv2DTranspose(128, (5, 5), strides = 2, padding = 'same', kernel_initializer = 'he_normal'))
    G.add(BatchNormalization())
    G.add(Activation('relu'))

    G.add(Conv2DTranspose(64, (5, 5), strides = 2, padding = 'same', kernel_initializer = 'he_normal'))
    G.add(BatchNormalization())
    G.add(Activation('relu'))

    G.add(Conv2DTranspose(3, (5, 5), strides = 2, padding = 'same', kernel_initializer = 'he_normal'))
    G.add(BatchNormalization())
    G.add(Activation('tanh'))

    return G

In [41]:
#   Build Discriminator
def build_discriminator(input_shape):
    D = Sequential()

    D.add(Conv2D(64, (5, 5), strides = 2, padding = 'same', input_shape = input_shape, kernel_initializer = 'he_normal'))
    D.add(BatchNormalization())
    D.add(LeakyReLU())

    D.add(Conv2D(128, (5, 5), strides = 2, padding = 'same', kernel_initializer = 'he_normal'))
    D.add(BatchNormalization())
    D.add(LeakyReLU())

    D.add(Conv2D(256, (5, 5), strides = 2, padding = 'same', kernel_initializer = 'he_normal'))
    D.add(BatchNormalization())
    D.add(LeakyReLU())

    D.add(Conv2D(512, (5, 5), strides = 2, padding = 'same', kernel_initializer = 'he_normal'))
    D.add(BatchNormalization())
    D.add(LeakyReLU())

    D.add(Flatten())
    D.add(Dense(1, kernel_initializer = 'he_normal'))
    D.add(Activation('sigmoid'))

    return D

In [42]:
def data_preprocess():
    (X_train, X_test), (_, _) = cifar10.load_data()
    X_train = X_train.astype(np.float32)
    X_train = 2*(X_train/255.0) - 1
    X_test = X_test.astype(np.float32)
    X_test = 2*(X_test/255.0) - 1
    return X_train, X_test

In [43]:
def plot_gen(images):
    (X_train, X_test), (_, _) = cifar10.load_data()
    height, width, channel = images.shape[1:]
    grid_size = (np.sqrt(images.shape[0])).astype('uint8')
    images = ((images + 1) / 2.0) * 255.0
    images = images.astype(np.uint8)
    images = (images.reshape(grid_size, grid_size, height, width, channel).transpose(0, 2, 1, 3, 4).reshape(grid_size*height, grid_size*width, channel))
    plt.imshow(images)
    plt.axis('off')
    plt.show()

In [44]:
def train(epochs, batch_size, plot_freq, z_dim):
    X_train, X_test = data_preprocess()
    z_fixed = np.random.rand(batch_size, z_dim)
    losses = {'D': [], 'G': []}

    G = build_generator(input_shape = (z_dim, ))
    D = build_discriminator(input_shape = (32, 32, 3))
    D.compile(optimizer = Adam(learning_rate = 1e-3, beta_1=0.5), loss='binary_crossentropy', metrics=['binary_accuracy'])

    D.trainable = False
    input = Input(shape = (z_dim, ))
    DCGAN = Model(inputs = input, outputs = D(G(input)))
    DCGAN.compile(optimizer = Adam(learning_rate = 1e-3, beta_1=0.5), loss='binary_crossentropy', metrics=['binary_accuracy'])

    dcgan_labels = np.zeros((2*batch_size, 1))
    dcgan_labels[:batch_size] = 1
    gen_labels = np.ones((batch_size, 1))
    iters = X_train.shape[0] // batch_size

    for e in tqdm_notebook(range(1, epochs + 1)):
        if e == 1 or e % plot_freq == 0:
            print('-'*30, f'Epoch {e}', '-'*30)
        for _ in range(iters):
            z_noise = np.random.rand(batch_size, z_dim)
            X_real = X_train[np.random.choice(X_train.shape[0], batch_size), :]

            #   Train Discriminator
            D.trainable = True
            img_gen = G.predict_on_batch(z_noise)
            imgs_gan = np.concatenate((X_real, img_gen))
            D_loss = D.train_on_batch(imgs_gan, dcgan_labels)

            #   Train Generator
            D.trainable = False
            G_loss = DCGAN.train_on_batch(z_noise, gen_labels)

            #   Loss
            losses['G'].append(G_loss)
            losses['D'].append(D_loss)
        if e == 1 or e % plot_freq == 0:
            plot_gen(G.predict(z_fixed))

In [ ]:
train(epochs = 200, batch_size = 36, plot_freq = 10, z_dim = 100)